<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#cuda" data-toc-modified-id="cuda-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>cuda</a></span></li><li><span><a href="#Utility" data-toc-modified-id="Utility-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utility</a></span></li><li><span><a href="#Params" data-toc-modified-id="Params-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Params</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

sns.set_style("whitegrid", {'axes.grid' : False})

# from tqdm.notebook import tqdm
from tqdm.auto import tqdm

## cuda

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


# Train

## Utility

In [4]:
def get_embeddings(model, loader):
    embeddings = torch.zeros((1, EMBEDDING_DIM))[1:].to(device)
    targets = torch.zeros((1))[1:].to(device)
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            new_emb = model(images)
            targets = torch.cat((targets, labels))
            embeddings = torch.cat((embeddings, new_emb))
    embeddings = embeddings.cpu()
    targets = targets.cpu()
    return embeddings, targets

In [5]:
def visualize_embeddings(model, loader):
    embeddings, targets = get_embeddings(model, loader)
    X = embeddings.cpu().numpy()
    y = targets.cpu().numpy()
    df = pd.DataFrame(X, columns=['x', 'y'])
    df['digit'] = y
    df['digit'] = df['digit'].astype(int).astype(str)
    
    fig, axs = plt.subplots(figsize=(8, 8))

    sns.scatterplot(
        x='x', y='y', hue='digit',
        alpha=0.5,
        data=df
    );
    
    
    return fig

In [6]:
def save_embedding_img(epoch, model, loader, accuracy=0, SAMPLE_IMAGES_DIR="mnist_embeddings/cosine_similarity"):
    fig = visualize_embeddings(model, loader)
    plt.title(f'epoch: {epoch:04}; accuracy: {accuracy:.3f}')
    plt.legend(loc="upper right")
    fig.tight_layout();
    
    fig.savefig(f'{SAMPLE_IMAGES_DIR}/img-{epoch:04}.png')
    plt.close(fig)

In [7]:
def test(model, trainloader, testloader):
    train_embeddings, train_labels = get_embeddings(model, trainloader)
    test_embeddings, test_labels = get_embeddings(model, testloader)
    accuracies = accuracy_calculator.get_accuracy(test_embeddings.numpy(), 
                                                train_embeddings.numpy(),
                                                test_labels.numpy(),
                                                train_labels.numpy(),
                                                False)
    return accuracies

In [8]:
def train(model, loss_func, optimizer, scheduler, trainloader, testloader, NUM_EPOCHS=5):
    losses = []
    test_accuracies = []
    
    for epoch in tqdm(range(NUM_EPOCHS)):
        model.train()
        epoch_losses = []
        for batch_idx, (images, labels) in tqdm(enumerate(trainloader)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            indices_tuple = mining_func(outputs, labels)
            loss = loss_func(outputs, labels, indices_tuple)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(float(loss.item()))
        
        loss = torch.tensor(epoch_losses).mean()
        losses.append(loss)
        
        accuracies = test(model, trainloader, testloader)
        test_acc = accuracies["precision_at_1"]
        test_accuracies.append(test_acc)
        
        scheduler.step(test_acc)
        
        save_embedding_img(epoch, model, testloader, accuracy=test_acc)
        
        print(f'epoch: {epoch}; loss: {loss}',
              f'test acc: {test_acc}')
    return losses, test_accuracies

## Params

In [9]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from pathlib import Path

In [10]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers, samplers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [11]:
BATCH_SIZE = 100
LR = 1e-3
SCHEDULER_EPOCHS = 2
EMBEDDING_DIM = 2

SAMPLES_PER_CLASS = 10 # drawing 16 classes per batch

In [12]:
torch.manual_seed(0);

## Data

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])

dataset = torchvision.datasets.MNIST(
    root='../data/mnist_data',
    train=True,
    download=True,
    transform=transform,
)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=96,
    shuffle=False, num_workers=4,
    pin_memory=True,
)

Compute mean, std

don't run if same dataset

In [14]:
def compute_mean_std(loader):
    mean = 0.0
    for images, _ in tqdm(loader):
        batch_samples = images.size(0) 
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)

    mean = mean / len(loader.dataset)

    var = 0.0
    for images, _ in tqdm(loader):
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        var += ((images - mean.unsqueeze(1))**2).sum([0,2])

        
    image_shape = torch.tensor(next(iter(loader))[0][0][0].shape).prod()
    
    std = torch.sqrt(var / (len(loader.dataset)*image_shape))
    return mean, std

In [15]:
# mean, std = compute_mean_std(dataloader)

In [16]:
mean = [0.1307, 0.1307, 0.1307]
std = [0.3081, 0.3081, 0.3081]

In [17]:
mean, std

([0.1307, 0.1307, 0.1307], [0.3081, 0.3081, 0.3081])

https://discuss.pytorch.org/t/grayscale-to-rgb-transform/18315/8

In [18]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean, std),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.MNIST(
    root='../data/mnist_data',
    train=True,
    download=True,
    transform=transform,
)

testset = torchvision.datasets.MNIST(
    root='../data/mnist_data',
    train=False, # test
    download=True,
    transform=transform,
)

In [19]:
train_targets = trainset.targets
test_targets = testset.targets

train_sampler = samplers.MPerClassSampler(
    train_targets,
    SAMPLES_PER_CLASS,
    batch_size=BATCH_SIZE,
    length_before_new_iter=BATCH_SIZE * 200  # 200 batches per epoch
)

In [20]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=BATCH_SIZE,
    num_workers=4,
    pin_memory=True,
    sampler=train_sampler,
)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=BATCH_SIZE,
    shuffle=False, num_workers=4,
    pin_memory=True,
)

## Model

In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [22]:
def update_model(model):
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, EMBEDDING_DIM)
    return model

In [23]:
torch.manual_seed(0);

In [24]:
torch.cuda.empty_cache()

In [25]:
model = torch.hub.load('pytorch/vision:v0.6.0',
                       'resnext50_32x4d',
                       pretrained=False)

Using cache found in /home/th/.cache/torch/hub/pytorch_vision_v0.6.0


In [26]:
# for (name, layer) in model._modules.items():
#     print((name, layer))

In [27]:
model = update_model(model)

In [28]:
model.to(device);

In [29]:
import gc
gc.collect()

80

In [30]:
dataiter = iter(dataloader)
images, labels = dataiter.next()
img_gpu = images.to(device)
with torch.no_grad():
    print(model(img_gpu).shape)
del img_gpu

torch.Size([96, 2])


In [31]:
params = [p for p in model.parameters() if p.requires_grad]

In [32]:
optimizer = optim.Adam(params, lr=LR)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=SCHEDULER_EPOCHS,
)

In [33]:
# distance = distances.CosineSimilarity()
distance = distances.DotProductSimilarity()
reducer = reducers.ThresholdReducer(low = 0)
loss_func = losses.TripletMarginLoss(margin = 0.2, distance = distance, reducer = reducer)
mining_func = miners.TripletMarginMiner(margin = 0.2, distance = distance, type_of_triplets = "semihard")
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)

In [34]:
losses, test_accuracies = train(
    model, loss_func, optimizer, scheduler,
    trainloader, testloader,
    NUM_EPOCHS=20
)

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.plot(losses, label='train loss')
# plt.plot(val_losses, label='validation loss')
plt.legend();

In [ ]:
# plt.plot(accuracy, label='train accuracy')
plt.plot(test_accuracies, label='validation accuracy')
plt.legend();

In [ ]:
!mkdir -p models

In [ ]:
MODEL_PATH = Path('./models/mnist_ml.pth')

In [ ]:
torch.save(model.state_dict(), MODEL_PATH)